In [1]:
import pymysql
import json

# Read JSON data from file
with open('model_output.json') as f:
    data = json.load(f)

In [2]:
# Connect to MySQL
connection = pymysql.connect(host='localhost', user='root', password='root@23', db='model', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
print("Connection to MySQL database established successfully.")

Connection to MySQL database established successfully.


# model DB auto increment

In [3]:
# Connect to MySQL
connection = pymysql.connect(host='localhost', user='root', password='root@23', db='model', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Insert data into `workflow` table
        for i, workflow_data in data.items():
            workflow_sql = "INSERT INTO workflow (workflow_name, project, workflow_description) VALUES (%s, %s, %s)"
            cursor.execute(workflow_sql, (workflow_data['workflow_name'], workflow_data['project'], workflow_data['workflow_description']))
            workflow_id = cursor.lastrowid
            
            # Insert data into `task` table
            for j, task_data in workflow_data['tasks'].items():
                task_sql = "INSERT INTO task (workflow_id, task_type, name, task_name, execution_order) VALUES (%s, %s, %s, %s, %s)"
                cursor.execute(task_sql, (workflow_id, task_data['task_type'], task_data['name'], task_data['task_name'], task_data['execution_order']))
                task_id = cursor.lastrowid
                
                # Insert data into `mapping` table
                mapping_data = task_data['mapping']
                if not mapping_data:
                    continue
                # print(f"TAsk id: {j}")

                mapping_sql = "INSERT INTO mapping (task_id, mapping_name, mapping_description) VALUES (%s, %s, %s)"
                cursor.execute(mapping_sql, (task_id, mapping_data['mapping_name'], mapping_data['mapping_description']))
                mapping_id = cursor.lastrowid
                

                # Insert data into `instance` table
                for k, instance_data in mapping_data['instances'].items():
                    # print(f"Instance id: {k}")
                    instance_sql = "INSERT INTO instance (mapping_id, instance_name, transformation_type, transformation_name, execution_order) VALUES (%s, %s, %s, %s, %s)"
                    cursor.execute(instance_sql, (mapping_id, instance_data['instance_name'], instance_data['transformation_type'], instance_data['transformation_name'], instance_data['execution_order']))
                    instance_id = cursor.lastrowid
                    
                    # Insert data into `field` table
                    for field_data in instance_data['fields']:
                        field_sql = "INSERT INTO field (instance_id, field_name, field_type, is_input, is_output, is_lookup) VALUES (%s, %s, %s, %s, %s, %s)"
                        # cursor.execute(field_sql, (instance_id, field_data['field_name'], field_data['field_type'], "1" if field_data['is_input']=="TRUE" else "0", "1" if field_data['is_output']=="TRUE" else "0", "1" if field_data['is_lookup']=="TRUE" else "0"))
                        cursor.execute(field_sql, (instance_id, field_data['field_name'], field_data['field_type'], field_data['port_values']['is_input'], field_data['port_values']['is_output'], field_data['port_values']['is_lookup']))
                        

    # Commit the transaction
    connection.commit()

except Exception as e:
    print(f"Error: {e}")
    # Rollback in case of any error
    connection.rollback()

finally:
    connection.close()
